In [ ]:
import numpy as np
from collections import defaultdict
import random
import Weekdays
import Professor
import Subject
import Students
import Room
import University

class Initializing:

    def __init__(self, dataframe, university_name):
      self.df = dataframe
      self.subject_name = dataframe['nazwa_przedmiotu']
      self.university= University()

      name_ID = defaultdict(int)
      name_letter_index = []
      df_exploded = self.df.explode('realizatorzy_przedmiotu')
      teachers = np.unique(np.concatenate((df_exploded['realizatorzy_przedmiotu'], ['realizatorzy_przedmiotu'])))
      teachers = teachers[teachers != 'realizatorzy_przedmiotu']
      for i in teachers:
          name, surname = i.split()
          letter = name[0].upper()
          how_much = np.sum(teachers == i)

          if how_much > 1:
              ID = name_ID[surname]
              surname_ID = f'{surname}_{letter}{ID}'
              name_ID[surname] += 1
          else:
              surname_ID = f'{surname}_{letter}'
              name_letter_index.append(surname_ID)

      self.teachers = name_letter_index

      #adding teachers
      for i in range(len(teachers)):
          self.teachers[i] = Professor(teachers[i])

      for i in  range(len(teachers)):
          self.university.add_professor(self.teachers[i].name)

      #adding greades
      for i in range(1,7):
          self.university.add_grade('Matematyka licencjat ' + str(i) ,i)

      #generate subjects from row
      classes = []
      for i in range(len(self.df)):
          name = self.subject_name[i]
          wyklad_hours = self.df['W'][i]/30
          cwiczenia_hours =self.df['Ć'][i]/30
          lab_hours =self.df['L'][i]/30
          proj_hours = self.df['P'][i]/30
          semin_hours = self.df['S'][i]/30
          yr = self.df['Semestr'][i]
          subjects = []
          while (wyklad_hours>=0.5):
            subjects.append(Subject(name + ' - W', self.university.professors[self.df['kierownik_przedmiotu'][i]], self.university.grades['Matematyka licencjat ' + str(yr)]))
            wyklad_hours -= 1
          while (lab_hours>=0.5):
            subjects.append(Subject(name + ' - L', self.university.professors[self.df['realizatorzy_przedmiotu'][i][0]], self.university.grades['Matematyka licencjat ' + str(yr)], classtype = 'labolatories'))
            lab_hours -= 1
          while (cwiczenia_hours>=0.5):
            if self.subject_name[i] == 'Wychowanie fizyczne':
              subjects.append(Subject(name, self.university.professors[self.df['realizatorzy_przedmiotu'][i][0]], self.university.grades['Matematyka licencjat ' + str(yr)], classtype = 'pe'))
          #  elif self.subject_name == 'Język obcy':
            #  subjects.append(Subject(name, self.university.professors[self.df['realizatorzy_przedmiotu'][i][0]], self.university.grades['Matematyka licencjat ' + str(yr)], classtype = 'english'))
            else:
              subjects.append(Subject(name + ' - C', self.university.professors[self.df['realizatorzy_przedmiotu'][i][0]], self.university.grades['Matematyka licencjat ' + str(yr)]))
            cwiczenia_hours -= 1
          while (proj_hours>=0.5):
            subjects.append(Subject(name + ' - P', self.university.professors[self.df['realizatorzy_przedmiotu'][i][0]], self.university.grades['Matematyka licencjat ' + str(yr)]))
            proj_hours -= 1
          while (semin_hours>=0.5):
            subjects.append(Subject(name + ' - S', self.university.professors[self.df['realizatorzy_przedmiotu'][i][0]], self.university.grades['Matematyka licencjat ' + str(yr)]))
            semin_hours -= 1

          classes.append(subjects)

      #adding rooms to University
      self.lab = []
      for i in range(1,4):
        self.lab.append('5' + str(i))
        self.university.add_room('5' + str(i), classtype = 'labolatories')
      self.lab_key = {key: self.university.rooms[key] for key in self.lab}

      self.labolatories = [self.lab_key[i] for i in self.lab]

      self.other = []
      for i in range(1,7):
        self.other.append('F' + str(i))
        self.university.add_room('F' + str(i), classtype='standard')
      self.other_key = {key: self.university.rooms[key] for key in self.other}

      self.standard = [self.other_key[i] for i in self.other]

      self.wf = []
      self.wf.append('Zatoka Sportu')
      self.wf.append('Nad Undergrondem')
      self.university.add_room('Zatoka Sportu', classtype='pe')
      self.university.add_room('Nad Undergrondem', classtype='pe')
      self.wf_key = {key: self.university.rooms[key] for key in self.wf}
      self.pe = [self.wf_key[i] for i in self.wf]

      self.eng = []
      self.eng.append('Centrum Językowe sala 1')
      self.eng.append('Centrum Językowe sala 2')
      self.university.add_room('Centrum Językowe sala 1', classtype='english')
      self.university.add_room('Centrum Językowe sala 2', classtype='english')
      self.eng_key = {key: self.university.rooms[key] for key in self.eng}
      self.lang = [self.wf_key[i] for i in self.wf]

      self.subject = subjects
      for i in range(len(self.df)):
          self.df['nazwa_przedmiotu'][i] = classes[i]


      #making random plan for all students and profesors at university
      for k in range(1,7):
        sub_dane=self.df[self.df['Semestr'] == k]
        sub_dane.reset_index(inplace=True, drop=True)
        for j in range(len(sub_dane)):
          for i in range(len(sub_dane['nazwa_przedmiotu'][j])):
            warunek = False
            while not warunek:
              if sub_dane['nazwa_przedmiotu'][j][i].classtype == 0: #other
                x = random.randint(0,4)
                y = random.randint(0,4)
                s = random.randint(0,5)
                if pd.isnull(sub_dane['nazwa_przedmiotu'][j][i].lecturer.class_assignment.iloc[x][y]) and pd.isnull(self.university.grades['Matematyka licencjat ' + str(k)].class_assignment.iloc[x][y]) and pd.isnull(self.standard[s].class_assignment.iloc[x][y]):
                  self.university.grades['Matematyka licencjat ' + str(k)].class_assignment.iloc[x][y] = sub_dane['nazwa_przedmiotu'][j][i]
                  self.university.grades['Matematyka licencjat ' + str(k)].class_assignment.iloc[x][y].time = x
                  self.university.grades['Matematyka licencjat ' + str(k)].class_assignment.iloc[x][y].day = y
                  sub_dane['nazwa_przedmiotu'][j][i].lecturer.class_assignment.iloc[x][y] = sub_dane['nazwa_przedmiotu'][j][i]
                  sub_dane['nazwa_przedmiotu'][j][i].lecturer.class_assignment.iloc[x][y].time = x
                  sub_dane['nazwa_przedmiotu'][j][i].lecturer.class_assignment.iloc[x][y].day = y
                  self.standard[s].class_assignment.iloc[x][y] = sub_dane['nazwa_przedmiotu'][j][i]
                  sub_dane['nazwa_przedmiotu'][j][i].add_room(self.standard[s].name)
                  warunek = True
                else:
                  warunek = False
              if sub_dane['nazwa_przedmiotu'][j][i].classtype == 1: #lab
                x = random.randint(0,4)
                y = random.randint(0,4)
                s = random.randint(0,2)
                if pd.isnull(sub_dane['nazwa_przedmiotu'][j][i].lecturer.class_assignment.iloc[x][y]) and pd.isnull(self.university.grades['Matematyka licencjat ' + str(k)].class_assignment.iloc[x][y]) and pd.isnull(self.labolatories[s].class_assignment.iloc[x][y]):
                  self.university.grades['Matematyka licencjat ' + str(k)].class_assignment.iloc[x][y] = sub_dane['nazwa_przedmiotu'][j][i]
                  self.university.grades['Matematyka licencjat ' + str(k)].class_assignment.iloc[x][y].time = x
                  self.university.grades['Matematyka licencjat ' + str(k)].class_assignment.iloc[x][y].day = y
                  sub_dane['nazwa_przedmiotu'][j][i].lecturer.class_assignment.iloc[x][y] = sub_dane['nazwa_przedmiotu'][j][i]
                  sub_dane['nazwa_przedmiotu'][j][i].lecturer.class_assignment.iloc[x][y].time = x
                  sub_dane['nazwa_przedmiotu'][j][i].lecturer.class_assignment.iloc[x][y].day = y
                  self.labolatories[s].class_assignment.iloc[x][y] = sub_dane['nazwa_przedmiotu'][j][i]
                  sub_dane['nazwa_przedmiotu'][j][i].add_room(self.labolatories[s].name)
                  warunek = True
                else:
                  warunek = False
              if sub_dane['nazwa_przedmiotu'][j][i].classtype == 2: #physical education
                x = random.randint(0,4)
                y = random.randint(0,4)
                s = random.randint(0,0)
                if pd.isnull(sub_dane['nazwa_przedmiotu'][j][i].lecturer.class_assignment.iloc[x][y]) and pd.isnull(self.university.grades['Matematyka licencjat ' + str(k)].class_assignment.iloc[x][y]) and pd.isnull(self.pe[s].class_assignment.iloc[x][y]):
                  self.university.grades['Matematyka licencjat ' + str(k)].class_assignment.iloc[x][y] = sub_dane['nazwa_przedmiotu'][j][i]
                  self.university.grades['Matematyka licencjat ' + str(k)].class_assignment.iloc[x][y].time = x
                  self.university.grades['Matematyka licencjat ' + str(k)].class_assignment.iloc[x][y].day = y
                  sub_dane['nazwa_przedmiotu'][j][i].lecturer.class_assignment.iloc[x][y] = sub_dane['nazwa_przedmiotu'][j][i]
                  sub_dane['nazwa_przedmiotu'][j][i].lecturer.class_assignment.iloc[x][y].time = x
                  sub_dane['nazwa_przedmiotu'][j][i].lecturer.class_assignment.iloc[x][y].day = y
                  self.pe[s].class_assignment.iloc[x][y] = sub_dane['nazwa_przedmiotu'][j][i]
                  sub_dane['nazwa_przedmiotu'][j][i].add_room(self.pe[s].name)
                  warunek = True
                else:
                  warunek = False
              if sub_dane['nazwa_przedmiotu'][j][i].classtype == 3: #language center
                x = random.randint(0,4)
                y = random.randint(0,4)
                s = random.randint(0,0)
                if pd.isnull(sub_dane['nazwa_przedmiotu'][j][i].lecturer.class_assignment.iloc[x][y]) and pd.isnull(self.university.grades['Matematyka licencjat ' + str(k)].class_assignment.iloc[x][y]) and pd.isnull(self.lang[s].class_assignment.iloc[x][y]):
                  self.university.grades['Matematyka licencjat ' + str(k)].class_assignment.iloc[x][y] = sub_dane['nazwa_przedmiotu'][j][i]
                  self.university.grades['Matematyka licencjat ' + str(k)].class_assignment.iloc[x][y].time = x
                  self.university.grades['Matematyka licencjat ' + str(k)].class_assignment.iloc[x][y].day = y
                  sub_dane['nazwa_przedmiotu'][j][i].lecturer.class_assignment.iloc[x][y] = sub_dane['nazwa_przedmiotu'][j][i]
                  sub_dane['nazwa_przedmiotu'][j][i].lecturer.class_assignment.iloc[x][y].time = x
                  sub_dane['nazwa_przedmiotu'][j][i].lecturer.class_assignment.iloc[x][y].day = y
                  self.lang[s].class_assignment.iloc[x][y] = sub_dane['nazwa_przedmiotu'][j][i]
                  sub_dane['nazwa_przedmiotu'][j][i].add_room(self.lang[s].name)
                  warunek = True
                else:
                  warunek = False

    def plan_to_list(self):

      school = list()
      categ = list()
      lab = []
      wyk = []
      ang = []
      wf = []

      # creating a list of classes taking place in rooms (first 5 elements of this list are classes that take place in specific room on Monday and the last 5 elements are classes that take place on Friday)
      for room in self.university.rooms:  #room
          lista=list()
          for i in range(5): #day
              for j in range(5): #time
                  lista.append(self.university.rooms[room].class_assignment.iloc[j,i])
          self.university.rooms[room].list=lista

      for room in self.university.rooms:  #room
          school.append(self.university.rooms[room].list)
          categ.append(self.university.rooms[room].category)

      for i in range(len(categ)):
          if categ[i] == 1:
            lab.append(school[i])
          elif categ[i] == 0:
            wyk.append(school[i])
          elif categ[i] == 2:
            wf.append(school[i])
          else:
            ang.append(school[i])

      plan_list = [lab,wyk,ang,wf]

      return plan_list